In [2]:
!pip install langchain langchain-community langchain-groq langchain-pinecone pymupdf langchain_core
!pip install pypdf
!pip install tiktoken
!pip install sentence-transformers
!pip install langchain-experimental

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 33.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.9/24.9 MB 72.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 59.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.8/85.8 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 500.5/500.5 kB 44.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 106.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.6/587.6 kB 38.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.3/259.3 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 6.1 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 330.6/330.6 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.1/210.1 kB 5.2 MB/s eta 0:00:00


In [1]:
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_groq import ChatGroq
from langchain_pinecone import PineconeVectorStore
from langchain_core.prompts import ChatPromptTemplate
import os

In [2]:
!mkdir pdfs

mkdir: cannot create directory ‘pdfs’: File exists


In [28]:
from langchain_community.document_loaders import PyMuPDFLoader

# Path to the specific problematic PDF file
file_path = "/content/pdfs/Paediatrics CS edited.pdf"

try:
    # Initialize PyMuPDFLoader with the specific file
    pdf_loader = PyMuPDFLoader(file_path)
    # Load the document
    mu_data = pdf_loader.load()
    print(f"Successfully loaded {len(mu_data)} pages from {file_path}")
    # Display the first document's content (or a snippet)
    if mu_data:
        print("\nFirst page content snippet:")
        print(mu_data[0].page_content[:500]) # Print first 500 characters of the first page
except Exception as e:
    print(f"Error loading PDF with PyMuPDFLoader: {e}")
    print("It seems the PDF file might be severely corrupted or not a valid PDF document.")
    print("Please ensure the PDF file is valid and not password-protected or encrypted.")

Successfully loaded 316 pages from /content/pdfs/Paediatrics CS edited.pdf

First page content snippet:
Year 
2020 
Paediatrics Slides 
Compiled 
Courtesy and acknowledgement to all our Lecturers form 
making their slide available for the purpose of learning.


In [4]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)

In [5]:
text_chunks = text_splitter.split_documents(mu_data)

In [6]:
print(text_chunks[51].page_content)

Apical DM (MS) 
DM in aortic area + collapsing pulses 
(AI) 
Signs of heart failure 
 
Investigations 
CXR 
    Cardiomegaly -LA, LV, RA, RV 
    Mitralised left heart border 
    Pulmonary congestion 
ECG 
    P mitrale (LAH) 
     PR interval 
    LVH +RVH 
 
 
Figure 1  CXR – LVH, LAH, inc. PVM


In [7]:
embedding = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

/tmp/ipython-input-1019514392.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [8]:
embedding.embed_query("How are you")

[-0.03136896714568138,
 0.03783046081662178,
 0.07630817592144012,
 0.045699622482061386,
 -0.0012047516647726297,
 -0.0747690349817276,
 0.08157838135957718,
 0.010209308005869389,
 -0.11220552772283554,
 0.04073430225253105,
 -0.04470575228333473,
 -0.0090286610648036,
 -0.022976186126470566,
 -0.009182579815387726,
 0.00713356863707304,
 -0.03529725968837738,
 0.07896504551172256,
 -0.0991559848189354,
 -0.12103831022977829,
 0.03257377818226814,
 -0.09918763488531113,
 0.031969573348760605,
 0.001739468309096992,
 0.0881587490439415,
 -0.026670122519135475,
 0.017091626301407814,
 -0.04135077819228172,
 -0.039181869477033615,
 0.035009175539016724,
 -0.07608215510845184,
 -0.06472422927618027,
 0.022676261141896248,
 -0.04956917464733124,
 -0.027594149112701416,
 -0.05726958438754082,
 -0.04053013399243355,
 0.01654020883142948,
 -0.1005965992808342,
 -0.0487324520945549,
 -0.022784588858485222,
 0.020854966714978218,
 -0.06302396208047867,
 -0.01944417878985405,
 -0.02730962820351

In [9]:
PINECONE_API_KEY = os.environ.get("PINECONE_API_KEY", "")
PINECONE_API_ENV = os.environ.get("PINECONE_API_ENV", "")

In [10]:
import os

index_name = "testing-index"

os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["PINECONE_API_ENV"] = PINECONE_API_ENV

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    embedding=embedding,
    index_name=index_name
)

In [11]:
docsearch

In [12]:
query = "what is the treatment for biliary atresia"

In [13]:
docs = docsearch.similarity_search(query)

In [14]:
docs

[Document(id='8b4a30ae-c89b-4362-a962-cbd6799bb84d', metadata={'author': 'Courtesy and acknowledgement to all our Lecturers form making their slide available for the purpose of learning.', 'creationDate': "D:20201209114854-08'00'", 'creationdate': '2020-12-09T11:48:54-08:00', 'creator': 'Microsoft® Word 2010', 'file_path': '/content/Paediatrics CS edited.pdf', 'format': 'PDF 1.5', 'keywords': '', 'modDate': "D:20250911081912+01'00'", 'moddate': '2025-09-11T08:19:12+01:00', 'page': 86.0, 'producer': 'Microsoft® Word 2010', 'source': '/content/Paediatrics CS edited.pdf', 'subject': 'Compiled', 'title': 'Paediatrics Slides', 'total_pages': 316.0, 'trapped': ''}, page_content='87 \n \nDUODENAL ATRESIA \nEpidemiology \n• Failure of recanalization \n• Incidence: 1 in 10,000 LB in Europe. \n• No familial incidence. \n• Associated anomalies - 50%. \n• Down’s syndrome - commonest chromosomal \nanomaly. \n \nPathology \n• Atresia/stenosis - 2nd part » 85%. \n• Rest - 15% \n \nClinical Features \

In [15]:
from langchain_core.stores import InMemoryStore
import uuid

In [21]:
llm = ChatGroq(
    model="openai/gpt-oss-120b",
    api_key=os.environ.get("GROQ_API_KEY", ""),
    temperature=0.5
)

In [22]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# 2. Setup the Retriever
# We convert your Pinecone VectorStore into a Retriever
retriever = docsearch.as_retriever(search_kwargs={"k": 10})

# 3. Define the Prompt
# This template tells the model exactly how to handle your medical slides
template = """
You are an expert medical assistant specializing in Paediatrics.
Use the following pieces of context (retrieved from medical slides) to answer the question.

If the answer is not in the context, say "I cannot find the answer in the provided slides."
Keep your answer concise and clinical.

Context:
{context}

Question:
{question}
"""

prompt = ChatPromptTemplate.from_template(template)

# 4. Define the Helper Function
# This combines the list of documents into a single string
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# 5. Build the Chain (The "Fix")
# This replaces RetrievalQA. It connects the steps manually:
# Retriever -> Format Docs -> Prompt -> LLM -> String Output
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

query = "what is the treatment for Biliary Atresia"

print(f"Question: {query}\n")

# .invoke() runs the whole pipeline
answer = rag_chain.invoke(query)

print("Answer:")
print(answer)

Question: what is the treatment for Biliary Atresia

Answer:
I cannot find the answer in the provided slides.


In [23]:
import sys
while True:
  user_input = input(f"Input Prompt: ")
  if user_input == "exit":
    print("Exiting")
    sys.exit()
  if user_input == "":
    continue
  result = rag_chain.invoke(user_input)
  print(f"Answer: {result}")

Input Prompt: Tell me about the management of guillain barre syndrome
Answer: I cannot find the answer in the provided slides.
Input Prompt: Tell me about the managementof guillain barre syndrome
Answer: I cannot find the answer in the provided slides.
Input Prompt: Tell me about the treatmentof guillain barre syndrome
Answer: **Treatment of Guillain‑Barré syndrome (GBS)**  

The primary goal is symptom control and prevention of complications. According to the slides, the therapeutic options include:

| Modality | Example(s) | Rationale |
|----------|------------|-----------|
| **Anticholinesterases** | Neostigmine, Pyridostigmine (oral, q.i.d. before meals) | Inhibit acetylcholine breakdown at the motor end‑plate, increasing neuromuscular transmission. |
| **Immunosuppressants** | Corticosteroids, Azathioprine, Cyclosporin | Suppress the immune response that mediates peripheral nerve demyelination. |
| **Plasmapheresis** | – | Removes circulating auto‑antibodies and inflammatory media

SystemExit: 

/usr/local/lib/python3.12/dist-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
